In [10]:
import time
import random
import numpy as np
import pandas as pd

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

from IPython.display import display
import ipywidgets as widgets

In [11]:
# サンプルデータ(ダミー)
pitches_colors = {
    'Four-seam Fastball': 'Black',
    'Two-seam Fastball': 'DimGray',
    'Cutter': 'CadetBlue',
    'Splitter': 'Coral',
    'Forkball': 'Crimson',
    'Curveball': 'Gold',
    'Slider': 'Indigo',
    'Slurve': 'LawnGreen',
    'Screwball': 'Orange',
    'Changeup': 'Orchid',
    'Palmball': 'SaddleBrown',
    'Circle Changeup': 'PaleVioletRed'
}

df = pd.DataFrame(np.random.randint(0, 100, (100, 2)))
df.columns = ['x', 'y']
df['speed'] = np.random.randint(80, 160, 100)
df['pitches'] = [random.choice(list(pitches_colors.keys())) for x in df.index]

In [12]:
# %matplotlib inline みたいな処理
output_notebook()

# グラフの初期化
opts = dict(plot_width=250, plot_height=250, min_border=0)
xy_range = df['x'].min(), df['x'].max()
p = figure(**opts, x_range=(xy_range), y_range=(xy_range))
p.line([20, 20, 80, 80, 20],
       [20, 80, 80, 20, 20],
       line_width=4, color="firebrick")

# 直近
size0 = df.loc[0, 'speed'] / 5
r = p.circle([df.loc[0, 'x']], [df.loc[0, 'y']],
             size=size0, line_color=None)
# 一投前
r1 = p.circle_x([df.loc[0, 'x']], [df.loc[0, 'y']],
                size=size0, line_color='Black')
# 二投前
r2 = p.circle_cross([df.loc[0, 'x']], [df.loc[0, 'y']],
                    size=size0, line_color='Black')

Loading BokehJS ...

In [13]:
def draw(n):
    def plot_pitches(i, renderer):
        x, y, speed, pitches = df.loc[i]
        renderer.data_source.data['x'] = [x]
        renderer.data_source.data['y'] = [y]
        renderer.glyph.size = speed / 5
        renderer.glyph.fill_color = pitches_colors[pitches]

    x, y, speed, pitches = df.loc[n]
    plot_pitches(n, r)
    p.title.text = '{}: {}km/h'.format(pitches, speed)

    if n > 1:
        plot_pitches(n - 2, r2)

    if n:
        plot_pitches(n - 1, r1)

    # handle=t がポイント、tで描画したセルをハンドリング
    push_notebook(handle=t)

In [6]:
# ipywidgetsのUI
slider = widgets.SelectionSlider(
    options=list(df.index.map(str)),
    value='0',
    description='num',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True, )


def on_value_change(change):
    n = int(change['new'])
    draw(n)


slider.observe(on_value_change, names='value')

In [7]:
# notebook_handle=True をつけることで、あとで書き換えられる
t = show(p, notebook_handle=True)

In [8]:
display(slider)

In [9]:
# 1秒毎にアニメーション
for x in df.index[:30]:
    draw(x)
    time.sleep(1)